In [135]:
# https://www.kaggle.com/fedesoriano/heart-failure-prediction

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import MinMaxScaler
from category_encoders import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold
from sklearn.metrics import f1_score, make_scorer

data = pd.read_csv("heart.csv")

data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


Tento dataset obsahuje následující sloupce:

Age - věk pacienta
Sex - pohlaví pacienta
ChestPainType - Typ bolesti na hrudi [TA, ATA, NAP, ASY] - typical, atypical (spojeno se srdečním svalem), non-anginal (bolest na hrudi, která nesouvisí se srdečním svalem) asymptotic - žádná bolest
RestingBP - Klidový krevní tlak 
Cholesterol - hladina cholesterolu 
FastingBS - hladina cukru na lačno (1 pokud je víc než 120 jinak 0)
RestingECG - EKG v klidu [Normal, ST, LVH] normal - normální, ST - abnormalita, LVH - pravděpodobně nebo definitivně hypertrofie (zvětšení) levé komory
MaxHR - Maximální tep. frekvence (60 - 202)
ExerciseAngina - Jestli je bolest vyvolaná při cvičení (Y - ano, N - ne)
Oldpeak - Hodnota jak moc je hodnota ST na grafu oproti baseline (pokud je záporná, tak jde pod ní, pokud je kladná, tak je nad ní)
ST_Slope - [Up, Flat, Down] - Stoupající, neměnící se, klesající 
HeartDisease: [0, 1] - 0 nemá, 1 má

Záznamy pocházejí z různých klinik a celkově je záznamů 918. Našim cílem je naučit náš program na základě všech sloupců předpovědět, jestli dotyčný trpí nebo netrpí srdeční chorobou.

Nejdřív se podíváme jak vypadají jednotlivé sloupce a jestli chybí hodnoty

In [136]:
print(data.isnull().sum())

print(data.describe())

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64
              Age   RestingBP  Cholesterol   FastingBS       MaxHR  \
count  918.000000  918.000000   918.000000  918.000000  918.000000   
mean    53.510893  132.396514   198.799564    0.233115  136.809368   
std      9.432617   18.514154   109.384145    0.423046   25.460334   
min     28.000000    0.000000     0.000000    0.000000   60.000000   
25%     47.000000  120.000000   173.250000    0.000000  120.000000   
50%     54.000000  130.000000   223.000000    0.000000  138.000000   
75%     60.000000  140.000000   267.000000    0.000000  156.000000   
max     77.000000  200.000000   603.000000    1.000000  202.000000   

          Oldpeak  HeartDisease  
count  918.000000    918.000000  
mean     0.887364      0.553377  
std      1.066

Žádné hodnoty nechybí, tak se pustíme do úpravy datasetu

Age - Použiju min-max scaler pro hezčí rozdělení hodnot
Sex - tam použiju one hot encoding 
ChestPainType - tam použiju OneHotEncoding proto, abych se vyvaroval tomu dávat pořadí a 4 stringové typy potřebujeme převést na čiselnou reprezentaci
RestingBP - pro jistotu bych použil min-max scaler, abychom měli jistotu, že to bude hezky rozprostřené
cholesterol - to samé, min-max scaler
FastingBS - tam máme rovnou bool hodnoty, není co měnit
RestingECG - OneHotEncoding ze stejného důvodu co ChestPainType
MaxHR - min-max scaler
ExerciseAngina - tam bych převedl Y-N na 1 a 0 ručně pomocí list comprehension
Oldpeak - tam bych použil min-max scaler, aby hodnoty byly hezčí, aktuálně na 1. pohled mají velké rozsahy
ST_Slope - OneHotEncoding
HeartDisease - není co měnit, výsledný sloupec s bool hodnotama


In [137]:

data["ExerciseAngina"] = [0 if x == "N" else 1 for x in data["ExerciseAngina"]]

hot_encoder = OneHotEncoder(cols=["Sex", "ChestPainType", "RestingECG", "ST_Slope"])
data = hot_encoder.fit_transform(data)

min_max_scaler = MinMaxScaler()
data[["Age", "RestingBP", "MaxHR", "Cholesterol", "Oldpeak"]] = min_max_scaler.fit_transform(
    data[["Age", "RestingBP", "MaxHR", "Cholesterol", "Oldpeak"]])

data

,Age,Sex_1,Sex_2,ChestPainType_1,ChestPainType_2,ChestPainType_3,ChestPainType_4,RestingBP,Cholesterol,FastingBS,RestingECG_1,RestingECG_2,RestingECG_3,MaxHR,ExerciseAngina,Oldpeak,ST_Slope_1,ST_Slope_2,ST_Slope_3,HeartDisease
0,0.244898,1,0,1,0,0,0,0.70,0.479270,0,1,0,0,0.788732,0,0.295455,1,0,0,0
1,0.428571,0,1,0,1,0,0,0.80,0.298507,0,1,0,0,0.676056,0,0.409091,0,1,0,1
2,0.183673,1,0,1,0,0,0,0.65,0.469320,0,0,1,0,0.267606,0,0.295455,1,0,0,0
3,0.408163,0,1,0,0,1,0,0.69,0.354892,0,1,0,0,0.338028,1,0.465909,0,1,0,1
4,0.530612,1,0,0,1,0,0,0.75,0.323383,0,1,0,0,0.436620,0,0.295455,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,0.346939,1,0,0,0,0,1,0.55,0.437811,0,1,0,0,0.507042,0,0.431818,0,1,0,1
914,0.816327,1,0,0,0,1,0,0.72,0.320066,1,1,0,0,0.570423,0,0.681818,0,1,0,1
915,0.591837,1,0,0,0,1,0,0.65,0.217247,0,1,0,0,0.387324,1,0.431818,0,1,0,1
916,0.591837,0,1,1,0,0,0,0.65,0.391376,0,0,0,1,0.802817,0,0.295455,0,1,0,1


Nyní vidíme, že všechny hodnoty byly převedeny pomocí one hot encodingu nebo min max scaleru a můžeme se pustit na výběr našeho měření úspěšnosti

In [138]:
print(data["HeartDisease"].value_counts(normalize=True))

HeartDisease
1    0.553377
0    0.446623
Name: proportion, dtype: float64


Jak je vidět, dataset je celkem hezky rozložený. Přesto bych ale zvolil F1 metriku, která je komplexnější než např. jen accuracy, ale lépe nám dokáže měřit falešně pozitivní (takže řekneme, že má nemoc, i když nemá) a falešně negativní (řekneme, že pacient je zdravý, i když není)

Jdeme se vrhnout na první učení a to za použití rozhodovacího stromu, ale nejdřív si ještě rozdělíme data na testovací a vybereme sloupce na trénování, přičemž vytvoříme testovací data 40% z původních (60/40)

In [139]:
X = data.drop("HeartDisease", axis=1)
y = data["HeartDisease"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=25)

tree_classifier = DecisionTreeClassifier(random_state=25)
tree_classifier.fit(x_train, y_train)

y_pred = tree_classifier.predict(x_test)

print(f"Přenost F1: {round(f1_score(y_test, y_pred), 3) * 100}%")

Přenost F1: 82.1%


Bez ladících parametrů máme 82% úspěšnost, to není špatné, ale věřím, že to jde i lépe. 

Jako 2. algoritmus zkusíme RandomForest

In [140]:
random_classifier = RandomForestClassifier(random_state=25)
random_classifier.fit(x_train, y_train)

y_pred = random_classifier.predict(x_test)

print(f"Přenost F1: {round(f1_score(y_test, y_pred), 3) * 100}%")

Přenost F1: 87.7%


Skoro 88% úspěšnost s random forest algoritmem, to je velmi hezké na to, že jsme nepoužili žádné ladění.

Jako 3. algoritmus se zvolíme Multi-layer-perceptron (MLP)

In [141]:
perceptron_classifier = MLPClassifier(random_state=25)
perceptron_classifier.fit(x_train, y_train)

y_pred = perceptron_classifier.predict(x_test)

print(f"Přenost F1: {round(f1_score(y_test, y_pred), 3) * 100}%")

Přenost F1: 87.8%


C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Opět skoro 88% úspěšnost, stejně jako s random forest algoritmem.

Nyní půjdeme na K-fold validaci, kde použijeme normální rozdělení, protože náš dataset je rozdělen na 45/55, což je skoro rovnoměrně rozdělené, takže není potřeba používat stratifikovanou verzi, která slouží hlavně v případech, kdy dataset není vyvážený.



In [142]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=25),
    "Random Forest": RandomForestClassifier(random_state=25),
    "MLP Classifier": MLPClassifier(random_state=25)
}

kf = KFold(n_splits=5, shuffle=True, random_state=25)
scorer = make_scorer(f1_score)

results = []

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=kf, scoring=scorer)
    results.append({
        "Algorithm": name,
        "Mean F1 Score": scores.mean(),
        "Std F1 Score": scores.std()
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


        Algorithm  Mean F1 Score  Std F1 Score
0   Decision Tree       0.804347      0.028688
1   Random Forest       0.890744      0.007141
2  MLP Classifier       0.872867      0.006359


C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Jak můžeme vidět, Nejlepší výsledky má random forest s nejlepší přesností a nejmenší odchylkou, tudíž se ho pokusíme vytunit co nejvíce v následující částí, kdy nám půjde hlavně o tuning parametrů.